## 机器视觉


<img src="data/logo.png" alt="Drawing" style="width: 300px;"/>


在这个课程项目中，我们会简单对用卷积神经网络实现的计算机视觉和相关知识进行介绍。虽然我们到现在都在使用CNN作文本处理，但是其实它最早还是用来完成计算机视觉任务的。


<img src="data/cnn_cv.png" alt="Drawing" style="width: 600px;"/>


## 配置

In [1]:
!pip install torch torchvision --upgrade -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install Pillow==4.1.1
!pip install image
!pip install matplotlib --upgrade

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
    100% |████████████████████████████████| 591.8MB 71kB/s 
Requirement already up-to-date: torchvision in /opt/conda/lib/python3.5/site-packages (0.2.1)
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 5.7MB 511kB/s 
  Found existing installation: Pillow 4.2.1
    Uninstalling Pillow-4.2.1:
      Successfully uninstalled Pillow-4.2.1
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 7.3MB 37kB/s 
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100

In [1]:
import os
from argparse import Namespace
import collections
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import re
import torch

In [2]:
# Set Numpy and PyTorch seeds
def set_seeds(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)
        
# Creating directories
def create_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

## 数据

我们先来获取一些数据。一个常用的计算机视觉分类数据集是 [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)。它包含了十种不同类型的图片数据。

In [3]:
# 这里我们只是使用tf来下载数据
import tensorflow as tf
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
# 加载数据
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
X = np.vstack([x_train, x_test])
y = np.vstack([y_train, y_test]).squeeze(1)
print ("x:", X.shape)
print ("y:", y.shape)

NameError: name 'tf' is not defined


每张图片的长和宽都是32像素，和三个色彩通道(就是RGB), 我们将它存在一个文件夹里，每种类型的图片都有自己的文件夹，文件夹名就是对应图片的类型。

In [6]:
!rm -rf cifar10_data

In [5]:
# Classes
classes = {0: 'plane', 1: 'car', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 
           6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

In [6]:
# Create image directories
data_dir = "cifar10_data"
os.mkdir(data_dir)
for _class in classes.values():
    os.mkdir(os.path.join(data_dir, _class))

In [7]:
# Save images for each class
for i, (image, label) in enumerate(zip(X, y)):
    _class = classes[label]
    im = Image.fromarray(image)
    im.save(os.path.join(data_dir, _class, "{0:02d}.png".format(i)))

NameError: name 'X' is not defined

In [10]:
# Visualize some samples
num_samples = len(classes)
for i, _class in enumerate(classes.values()):  
    for file in os.listdir(os.path.join(data_dir, _class)):
        if file.endswith(".png"):
            plt.subplot(1, num_samples, i+1)
            plt.title("{0}".format(_class))
            img = Image.open(os.path.join(data_dir, _class, file))
            plt.imshow(img)
            plt.axis("off")
            break

<Figure size 432x288 with 10 Axes>

## 分类

任务是为给出的图片分类，我们会构建一个结构最基础的CNN然后用它处理数据，输出预测的分类。

### 参数
处理图片数据时，不需要保存切分后的文件，我们只会从文件夹中读取他们。

In [11]:
args = Namespace(
    seed=1234,
    cuda=True,
    shuffle=True,
    data_dir="cifar10_data",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="cifar10_model",
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    num_epochs=10,
    early_stopping_criteria=5,
    learning_rate=1e-3,
    batch_size=128,
    num_filters=100,
    hidden_dim=100,
    dropout_p=0.1,
)

# Set seeds
set_seeds(seed=args.seed, cuda=args.cuda)

# Create save dir
create_dirs(args.save_dir)

# Expand filepaths
args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

Using CUDA: True


### 数据

In [12]:
# 将图片转换为Numpy Array
def img_to_array(fp):
    img = Image.open(fp)
    array = np.asarray(img, dtype="float32")
    return array

In [13]:
# 加载数据
data = []
for i, _class in enumerate(classes.values()):  
    for file in os.listdir(os.path.join(data_dir, _class)):
        if file.endswith(".png"):
            full_filepath = os.path.join(data_dir, _class, file)
            data.append({"image": img_to_array(full_filepath), "category": _class})

In [14]:
# 转化为 Dataframe
df = pd.DataFrame(data)
print ("Image shape:", df.image[0].shape)
df.head()

Image shape: (32, 32, 3)


,category,image
0,plane,"[[[105.0, 131.0, 164.0], [106.0, 132.0, 167.0]..."
1,plane,"[[[156.0, 160.0, 159.0], [156.0, 160.0, 160.0]..."
2,plane,"[[[91.0, 119.0, 141.0], [91.0, 121.0, 142.0], ..."
3,plane,"[[[82.0, 136.0, 186.0], [82.0, 135.0, 184.0], ..."
4,plane,"[[[115.0, 165.0, 224.0], [117.0, 167.0, 226.0]..."


In [15]:
by_category = collections.defaultdict(list)
for _, row in df.iterrows():
    by_category[row.category].append(row.to_dict())
for category in by_category:
    print ("{0}: {1}".format(category, len(by_category[category])))

cat: 6000
plane: 6000
car: 6000
deer: 6000
truck: 6000
frog: 6000
ship: 6000
dog: 6000
bird: 6000
horse: 6000


In [16]:
final_list = []
for _, item_list in sorted(by_category.items()):
    if args.shuffle:
        np.random.shuffle(item_list)
    n = len(item_list)
    n_train = int(args.train_size*n)
    n_val = int(args.val_size*n)
    n_test = int(args.test_size*n)

  # 给数据点一个切分属性
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  

    # Add to final list
    final_list.extend(item_list)

In [17]:
split_df = pd.DataFrame(final_list)
split_df["split"].value_counts()

train    42000
val       9000
test      9000
Name: split, dtype: int64

### 词汇表

In [18]:
class Vocabulary(object):
    def __init__(self, token_to_idx=None):

        # Token to index
        if token_to_idx is None:
            token_to_idx = {}
        self.token_to_idx = token_to_idx

        # Index to token
        self.idx_to_token = {idx: token \
                             for token, idx in self.token_to_idx.items()}

    def to_serializable(self):
        return {'token_to_idx': self.token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)

    def add_token(self, token):
        if token in self.token_to_idx:
            index = self.token_to_idx[token]
        else:
            index = len(self.token_to_idx)
            self.token_to_idx[token] = index
            self.idx_to_token[index] = token
        return index

    def add_tokens(self, tokens):
        return [self.add_token[token] for token in tokens]

    def lookup_token(self, token):
        return self.token_to_idx[token]

    def lookup_index(self, index):
        if index not in self.idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self.idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self.token_to_idx)

In [19]:
# Vocabulary instance
category_vocab = Vocabulary()
for index, row in df.iterrows():
    category_vocab.add_token(row.category)
print (category_vocab) # __str__
print (len(category_vocab)) # __len__
index = category_vocab.lookup_token("dog")
print (index)
print (category_vocab.lookup_index(index))

<Vocabulary(size=10)>
10
5
dog


### 序列化词汇表

In [20]:
from collections import Counter
import string

In [21]:
class SequenceVocabulary():
    def __init__(self, train_means, train_stds):
        
        self.train_means = train_means
        self.train_stds = train_stds
        
    def to_serializable(self):
        contents = {'train_means': self.train_means,
                    'train_stds': self.train_stds}
        return contents
    
    @classmethod
    def from_dataframe(cls, df):
        train_data = df[df.split == "train"]
        means = {0:[], 1:[], 2:[]}
        stds = {0:[], 1:[], 2:[]}
        for image in train_data.image:
            for dim in range(3):
                means[dim].append(np.mean(image[:, :, dim]))
                stds[dim].append(np.std(image[:, :, dim]))
        train_means = np.array((np.mean(means[0]), np.mean(means[1]), 
                                np.mean(means[2])), dtype="float64").tolist()
        train_stds = np.array((np.mean(stds[0]), np.mean(stds[1]), 
                               np.mean(stds[2])), dtype="float64").tolist()
            
        return cls(train_means, train_stds)
        
    def __str__(self):
        return "<SequenceVocabulary(train_means: {0}, train_stds: {1}>".format(
            self.train_means, self.train_stds)

In [22]:
# Create SequenceVocabulary instance
image_vocab = SequenceVocabulary.from_dataframe(split_df)
print (image_vocab) # __str__

<SequenceVocabulary(train_means: [125.40174102783203, 122.99897766113281, 114.00146484375], train_stds: [51.5576171875, 50.800132751464844, 51.22932052612305]>


### 向量化

In [23]:
class ImageVectorizer(object):
    def __init__(self, image_vocab, category_vocab):
        self.image_vocab = image_vocab
        self.category_vocab = category_vocab

    def vectorize(self, image):
        
        # Avoid modifying the actual df
        image = np.copy(image)
        
        # Normalize
        for dim in range(3):
            mean = self.image_vocab.train_means[dim]
            std = self.image_vocab.train_stds[dim]
            image[:, :, dim] = ((image[:, :, dim] - mean) / std)
            
        # Reshape frok (32, 32, 3) to (3, 32, 32)
        image = np.swapaxes(image, 0, 2)
        image = np.swapaxes(image, 1, 2)
                
        return image
    
    @classmethod
    def from_dataframe(cls, df):
        
        # Create class vocab
        category_vocab = Vocabulary()   
        for category in sorted(set(df.category)):
            category_vocab.add_token(category)
            
        # Create image vocab
        image_vocab = SequenceVocabulary.from_dataframe(df)
        
        return cls(image_vocab, category_vocab)

    @classmethod
    def from_serializable(cls, contents):
        image_vocab = SequenceVocabulary.from_serializable(contents['image_vocab'])
        category_vocab = Vocabulary.from_serializable(contents['category_vocab'])
        return cls(image_vocab=image_vocab, 
                   category_vocab=category_vocab)
    
    def to_serializable(self):
        return {'image_vocab': self.image_vocab.to_serializable(),
                'category_vocab': self.category_vocab.to_serializable()}

In [24]:
# Vectorizer instance
vectorizer = ImageVectorizer.from_dataframe(split_df)
print (vectorizer.image_vocab)
print (vectorizer.category_vocab)
image_vector = vectorizer.vectorize(split_df.iloc[0].image)
print (image_vector.shape)

<SequenceVocabulary(train_means: [125.40174102783203, 122.99897766113281, 114.00146484375], train_stds: [51.5576171875, 50.800132751464844, 51.22932052612305]>
<Vocabulary(size=10)>
(3, 32, 32)


### 数据集

In [25]:
from torch.utils.data import Dataset, DataLoader

In [26]:
class ImageDataset(Dataset):
    def __init__(self, df, vectorizer):
        self.df = df
        self.vectorizer = vectorizer

        # Data splits
        self.train_df = self.df[self.df.split=='train']
        self.train_size = len(self.train_df)
        self.val_df = self.df[self.df.split=='val']
        self.val_size = len(self.val_df)
        self.test_df = self.df[self.df.split=='test']
        self.test_size = len(self.test_df)
        self.lookup_dict = {'train': (self.train_df, self.train_size), 
                            'val': (self.val_df, self.val_size),
                            'test': (self.test_df, self.test_size)}
        self.set_split('train')

        # Class weights (for imbalances)
        class_counts = df.category.value_counts().to_dict()
        def sort_key(item):
            return self.vectorizer.category_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, df):
        train_df = df[df.split=='train']
        return cls(df, ImageVectorizer.from_dataframe(train_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, df, vectorizer_filepath):
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(df, vectorizer)

    def load_vectorizer_only(vectorizer_filepath):
        with open(vectorizer_filepath) as fp:
            return ImageVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self.vectorizer.to_serializable(), fp)

    def set_split(self, split="train"):
        self.target_split = split
        self.target_df, self.target_size = self.lookup_dict[split]

    def __str__(self):
        return "<Dataset(split={0}, size={1})".format(
            self.target_split, self.target_size)

    def __len__(self):
        return self.target_size

    def __getitem__(self, index):
        row = self.target_df.iloc[index]
        image_vector = self.vectorizer.vectorize(row.image)
        category_index = self.vectorizer.category_vocab.lookup_token(row.category)
        return {'image': image_vector, 
                'category': category_index}

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

    def generate_batches(self, batch_size, shuffle=True, drop_last=True, device="cpu"):
        dataloader = DataLoader(dataset=self, batch_size=batch_size, 
                                shuffle=shuffle, drop_last=drop_last)
        for data_dict in dataloader:
            out_data_dict = {}
            for name, tensor in data_dict.items():
                out_data_dict[name] = data_dict[name].to(device)
            yield out_data_dict

In [27]:
# Dataset instance
dataset = ImageDataset.load_dataset_and_make_vectorizer(split_df)
print (dataset) # __str__
input_ = dataset[10] # __getitem__
print (input_['image'].shape)
category = input_['category']
print (dataset.vectorizer.category_vocab.lookup_index(category))
print (dataset.class_weights)

<Dataset(split=train, size=42000)
(3, 32, 32)
bird
tensor([0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002,
        0.0002])


### 模型

In [28]:
import torch.nn as nn
import torch.nn.functional as F

In [29]:
class ImageModel(nn.Module):
    def __init__(self, num_hidden_units, num_classes, dropout_p):
        super(ImageModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5) # input_channels:3 , output_channels:10 (aka num filters)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.conv_dropout = nn.Dropout2d(dropout_p)
        self.fc1 = nn.Linear(20*5*5, num_hidden_units)
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(num_hidden_units, num_classes)

    def forward(self, x, apply_softmax=False):
          
        # 卷积层和池化
        z = self.conv1(x) # (N, 10, 28, 28)
        z = F.max_pool2d(z, 2) # (N, 10, 14, 14)
        z = F.relu(z)
        
        # 卷积层和池化
        z = self.conv2(z) # (N, 20, 10, 10)
        z = self.conv_dropout(z) 
        z = F.max_pool2d(z, 2) # (N, 20, 5, 5)
        z = F.relu(z)
        
        # 整平操作
        z = z.view(-1, 20*5*5)
        
        # 全连接层
        z = F.relu(self.fc1(z))
        z = self.dropout(z)
        y_pred = self.fc2(z)
        
        if apply_softmax:
            y_pred = F.softmax(y_pred, dim=1)
        return y_pred 
        

### 训练

In [30]:
import torch.optim as optim

In [31]:
class Trainer(object):
    def __init__(self, dataset, model, model_state_file, save_dir, device, 
                 shuffle, num_epochs, batch_size, learning_rate, 
                 early_stopping_criteria):
        self.dataset = dataset
        self.class_weights = dataset.class_weights.to(device)
        self.device = device
        self.model = model.to(device)
        self.save_dir = save_dir
        self.device = device
        self.shuffle = shuffle
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.loss_func = nn.CrossEntropyLoss(self.class_weights)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=self.optimizer, mode='min', factor=0.5, patience=1)
        self.train_state = {
            'stop_early': False, 
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'early_stopping_criteria': early_stopping_criteria,
            'learning_rate': learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': model_state_file}
    
    def update_train_state(self):

        # Verbose
        print ("[EPOCH]: {0:02d} | [LR]: {1} | [TRAIN LOSS]: {2:.2f} | [TRAIN ACC]: {3:.1f}% | [VAL LOSS]: {4:.2f} | [VAL ACC]: {5:.1f}%".format(
          self.train_state['epoch_index'], self.train_state['learning_rate'], 
            self.train_state['train_loss'][-1], self.train_state['train_acc'][-1], 
            self.train_state['val_loss'][-1], self.train_state['val_acc'][-1]))

        # Save one model at least
        if self.train_state['epoch_index'] == 0:
            torch.save(self.model.state_dict(), self.train_state['model_filename'])
            self.train_state['stop_early'] = False

        # Save model if performance improved
        elif self.train_state['epoch_index'] >= 1:
            loss_tm1, loss_t = self.train_state['val_loss'][-2:]

            # If loss worsened
            if loss_t >= self.train_state['early_stopping_best_val']:
                # Update step
                self.train_state['early_stopping_step'] += 1

            # Loss decreased
            else:
                # Save the best model
                if loss_t < self.train_state['early_stopping_best_val']:
                    torch.save(self.model.state_dict(), self.train_state['model_filename'])

                # Reset early stopping step
                self.train_state['early_stopping_step'] = 0

            # Stop early ?
            self.train_state['stop_early'] = self.train_state['early_stopping_step'] \
              >= self.train_state['early_stopping_criteria']
        return self.train_state
  
    def compute_accuracy(self, y_pred, y_target):
        _, y_pred_indices = y_pred.max(dim=1)
        n_correct = torch.eq(y_pred_indices, y_target).sum().item()
        return n_correct / len(y_pred_indices) * 100
  
    def run_train_loop(self):
        for epoch_index in range(self.num_epochs):
            self.train_state['epoch_index'] = epoch_index
      
            # Iterate over train dataset

            # initialize batch generator, set loss and acc to 0, set train mode on
            self.dataset.set_split('train')
            batch_generator = self.dataset.generate_batches(
                batch_size=self.batch_size, shuffle=self.shuffle, 
                device=self.device)
            running_loss = 0.0
            running_acc = 0.0
            self.model.train()

            for batch_index, batch_dict in enumerate(batch_generator):
                # zero the gradients
                self.optimizer.zero_grad()
                
                # compute the output
                y_pred = self.model(x=batch_dict['image'])
                
                # compute the loss
                loss = self.loss_func(y_pred, batch_dict['category'])
                loss_t = loss.item()
                running_loss += (loss_t - running_loss) / (batch_index + 1)

                # compute gradients using loss
                loss.backward()

                # use optimizer to take a gradient step
                self.optimizer.step()
                
                # compute the accuracy
                acc_t = self.compute_accuracy(y_pred, batch_dict['category'])
                running_acc += (acc_t - running_acc) / (batch_index + 1)

            self.train_state['train_loss'].append(running_loss)
            self.train_state['train_acc'].append(running_acc)

            # Iterate over val dataset

            # initialize batch generator, set loss and acc to 0, set eval mode on
            self.dataset.set_split('val')
            batch_generator = self.dataset.generate_batches(
                batch_size=self.batch_size, shuffle=self.shuffle, device=self.device)
            running_loss = 0.
            running_acc = 0.
            self.model.eval()

            for batch_index, batch_dict in enumerate(batch_generator):

                # compute the output
                y_pred = self.model(x=batch_dict['image'])

                # compute the loss
                loss = self.loss_func(y_pred, batch_dict['category'])
                loss_t = loss.to("cpu").item()
                running_loss += (loss_t - running_loss) / (batch_index + 1)

                # compute the accuracy
                acc_t = self.compute_accuracy(y_pred, batch_dict['category'])
                running_acc += (acc_t - running_acc) / (batch_index + 1)

            self.train_state['val_loss'].append(running_loss)
            self.train_state['val_acc'].append(running_acc)

            self.train_state = self.update_train_state()
            self.scheduler.step(self.train_state['val_loss'][-1])
            if self.train_state['stop_early']:
                break
          
    def run_test_loop(self):
        # initialize batch generator, set loss and acc to 0, set eval mode on
        self.dataset.set_split('test')
        batch_generator = self.dataset.generate_batches(
            batch_size=self.batch_size, shuffle=self.shuffle, device=self.device)
        running_loss = 0.0
        running_acc = 0.0
        self.model.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # compute the output
            y_pred = self.model(x=batch_dict['image'])

            # compute the loss
            loss = self.loss_func(y_pred, batch_dict['category'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = self.compute_accuracy(y_pred, batch_dict['category'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

        self.train_state['test_loss'] = running_loss
        self.train_state['test_acc'] = running_acc
    
    def plot_performance(self):
        # Figure size
        plt.figure(figsize=(15,5))

        # Plot Loss
        plt.subplot(1, 2, 1)
        plt.title("Loss")
        plt.plot(trainer.train_state["train_loss"], label="train")
        plt.plot(trainer.train_state["val_loss"], label="val")
        plt.legend(loc='upper right')

        # Plot Accuracy
        plt.subplot(1, 2, 2)
        plt.title("Accuracy")
        plt.plot(trainer.train_state["train_acc"], label="train")
        plt.plot(trainer.train_state["val_acc"], label="val")
        plt.legend(loc='lower right')

        # Save figure
        plt.savefig(os.path.join(self.save_dir, "performance.png"))

        # Show plots
        plt.show()
    
    def save_train_state(self):
        with open(os.path.join(self.save_dir, "train_state.json"), "w") as fp:
            json.dump(self.train_state, fp)

In [32]:
# Initialization
dataset = ImageDataset.load_dataset_and_make_vectorizer(split_df)
dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.vectorizer
model = ImageModel(num_hidden_units=args.hidden_dim, 
                   num_classes=len(vectorizer.category_vocab),
                   dropout_p=args.dropout_p)
print (model.named_modules)

<bound method Module.named_modules of ImageModel(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv_dropout): Dropout2d(p=0.1)
  (fc1): Linear(in_features=500, out_features=100, bias=True)
  (dropout): Dropout(p=0.1)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)>


In [33]:
# Train
trainer = Trainer(dataset=dataset, model=model, 
                  model_state_file=args.model_state_file, 
                  save_dir=args.save_dir, device=args.device,
                  shuffle=args.shuffle, num_epochs=args.num_epochs, 
                  batch_size=args.batch_size, learning_rate=args.learning_rate, 
                  early_stopping_criteria=args.early_stopping_criteria)
trainer.run_train_loop()

[EPOCH]: 00 | [LR]: 0.001 | [TRAIN LOSS]: 1.69 | [TRAIN ACC]: 38.6% | [VAL LOSS]: 1.47 | [VAL ACC]: 47.0%
[EPOCH]: 01 | [LR]: 0.001 | [TRAIN LOSS]: 1.39 | [TRAIN ACC]: 50.1% | [VAL LOSS]: 1.29 | [VAL ACC]: 53.9%
[EPOCH]: 02 | [LR]: 0.001 | [TRAIN LOSS]: 1.27 | [TRAIN ACC]: 54.9% | [VAL LOSS]: 1.19 | [VAL ACC]: 57.3%
[EPOCH]: 03 | [LR]: 0.001 | [TRAIN LOSS]: 1.19 | [TRAIN ACC]: 57.7% | [VAL LOSS]: 1.13 | [VAL ACC]: 60.7%
[EPOCH]: 04 | [LR]: 0.001 | [TRAIN LOSS]: 1.13 | [TRAIN ACC]: 60.0% | [VAL LOSS]: 1.11 | [VAL ACC]: 61.2%
[EPOCH]: 05 | [LR]: 0.001 | [TRAIN LOSS]: 1.08 | [TRAIN ACC]: 61.6% | [VAL LOSS]: 1.07 | [VAL ACC]: 62.5%
[EPOCH]: 06 | [LR]: 0.001 | [TRAIN LOSS]: 1.04 | [TRAIN ACC]: 63.2% | [VAL LOSS]: 1.06 | [VAL ACC]: 62.3%
[EPOCH]: 07 | [LR]: 0.001 | [TRAIN LOSS]: 1.02 | [TRAIN ACC]: 64.2% | [VAL LOSS]: 1.02 | [VAL ACC]: 63.8%
[EPOCH]: 08 | [LR]: 0.001 | [TRAIN LOSS]: 0.99 | [TRAIN ACC]: 65.5% | [VAL LOSS]: 1.01 | [VAL ACC]: 64.6%
[EPOCH]: 09 | [LR]: 0.001 | [TRAIN LOSS]: 0.96

In [34]:
# Plot performance
trainer.plot_performance()

<Figure size 1080x360 with 2 Axes>

In [35]:
# Test performance
trainer.run_test_loop()
print("Test loss: {0:.2f}".format(trainer.train_state['test_loss']))
print("Test Accuracy: {0:.1f}%".format(trainer.train_state['test_acc']))

Test loss: 0.98
Test Accuracy: 66.3%


In [36]:
# Save all results
trainer.save_train_state()


66% 左右的测试性能在 Cifar10 数据集上并不差，但是我们可以做的更好。


## Transfer learning
## 迁移学习

这一节里，我们即将使用在很多数据集上的性能都非常好的训练好的模型。我们会直接使用它的构架和权重来初始化模型，并用它训练。我们就冻结初始的卷积权重，然后调整后面的卷积和全连接层。
 
迁移学习之所以适用于这个场景，是因为最初的卷积层在处理图片中包含(不管他的类别是什么)的空间信息时表现十分优异。我们会用这些巨大的已训练好模型的特征提取器用在自己的数据集上。

In [37]:
from torchvision import models

In [38]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))
print (model_names)

['alexnet', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'inception_v3', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']


In [39]:
model_name = 'vgg19_bn'
vgg_19bn = models.__dict__[model_name](pretrained=True) # Set false to train from scratch
print (vgg_19bn.named_parameters)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /home/kesci/.torch/models/vgg19_bn-c79401a0.pth
100%|██████████| 574769405/574769405 [00:42<00:00, 13614017.36it/s]


<bound method Module.named_parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    

我们选择的 VGG模型 包含了 特征--`features` 和 分类器--`classifier`两个组件。`features`组件由卷积层和池化层组成，作为特征提取器。而 `classifier`组件由全连接层组成。我们将会冻结 `feature`组件绝大多的权重，然后为了CIFAR10分类任务自己设计全连接层。如果你想直接使用VGG而不是稍作修改，你可以在本地的 `/usr/local/lib/python3.X/dist-packages/torchvision/models` 访问默认的模型。 

In [40]:
class ImageModel(nn.Module):
    def __init__(self, feature_extractor, num_hidden_units, 
                 num_classes, dropout_p):
        super(ImageModel, self).__init__()
        
        # 训练好的特征提取器
        self.feature_extractor = feature_extractor
        
        # 全连接层权重
        self.classifier = nn.Sequential(
            nn.Linear(512, 250, bias=True),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(250, 100, bias=True),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(100, 10, bias=True),
            )

    def forward(self, x, apply_softmax=False):
          
        # 特征提取器
        z = self.feature_extractor(x)
        z = z.view(x.size(0), -1)
        
        # FC
        y_pred = self.classifier(z)

        if apply_softmax:
            y_pred = F.softmax(y_pred, dim=1)
        return y_pred 

In [41]:
# Initialization
dataset = ImageDataset.load_dataset_and_make_vectorizer(split_df)
dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.vectorizer
model = ImageModel(feature_extractor=vgg_19bn.features, 
                   num_hidden_units=args.hidden_dim,
                   num_classes=len(vectorizer.category_vocab), 
                   dropout_p=args.dropout_p)
print (model.named_parameters)

<bound method Module.named_parameters of ImageModel(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_

In [42]:
# 微调最后几层卷积层和全连接层
for i, param in enumerate(model.feature_extractor.parameters()):
    if i < 36:
        param.requires_grad = False
    else:
        param.requires_grad = True

In [43]:
# Train
trainer = Trainer(dataset=dataset, model=model, 
                  model_state_file=args.model_state_file, 
                  save_dir=args.save_dir, device=args.device,
                  shuffle=args.shuffle, num_epochs=args.num_epochs, 
                  batch_size=args.batch_size, learning_rate=args.learning_rate, 
                  early_stopping_criteria=args.early_stopping_criteria)
trainer.run_train_loop()

[EPOCH]: 00 | [LR]: 0.001 | [TRAIN LOSS]: 0.93 | [TRAIN ACC]: 71.7% | [VAL LOSS]: 0.60 | [VAL ACC]: 80.9%
[EPOCH]: 01 | [LR]: 0.001 | [TRAIN LOSS]: 0.57 | [TRAIN ACC]: 83.1% | [VAL LOSS]: 0.55 | [VAL ACC]: 83.3%
[EPOCH]: 02 | [LR]: 0.001 | [TRAIN LOSS]: 0.42 | [TRAIN ACC]: 87.7% | [VAL LOSS]: 0.55 | [VAL ACC]: 84.0%
[EPOCH]: 03 | [LR]: 0.001 | [TRAIN LOSS]: 0.33 | [TRAIN ACC]: 90.5% | [VAL LOSS]: 0.54 | [VAL ACC]: 84.7%
[EPOCH]: 04 | [LR]: 0.001 | [TRAIN LOSS]: 0.26 | [TRAIN ACC]: 92.7% | [VAL LOSS]: 0.53 | [VAL ACC]: 84.7%
[EPOCH]: 05 | [LR]: 0.001 | [TRAIN LOSS]: 0.19 | [TRAIN ACC]: 94.7% | [VAL LOSS]: 0.58 | [VAL ACC]: 85.0%
[EPOCH]: 06 | [LR]: 0.001 | [TRAIN LOSS]: 0.15 | [TRAIN ACC]: 95.8% | [VAL LOSS]: 0.64 | [VAL ACC]: 85.1%
[EPOCH]: 07 | [LR]: 0.001 | [TRAIN LOSS]: 0.08 | [TRAIN ACC]: 97.9% | [VAL LOSS]: 0.60 | [VAL ACC]: 86.6%
[EPOCH]: 08 | [LR]: 0.001 | [TRAIN LOSS]: 0.04 | [TRAIN ACC]: 98.9% | [VAL LOSS]: 0.72 | [VAL ACC]: 86.5%
[EPOCH]: 09 | [LR]: 0.001 | [TRAIN LOSS]: 0.03

In [44]:
# Plot performance
trainer.plot_performance()

<Figure size 1080x360 with 2 Axes>

In [45]:
# Test performance
trainer.run_test_loop()
print("Test loss: {0:.2f}".format(trainer.train_state['test_loss']))
print("Test Accuracy: {0:.1f}%".format(trainer.train_state['test_acc']))

Test loss: 0.72
Test Accuracy: 87.2%


In [46]:
# Save all results
trainer.save_train_state()

性能好了太多了！如果训练时间足够的话，最高可以达到 95% 的准确率 :)

## 预测

In [47]:
from pylab import rcParams
rcParams['figure.figsize'] = 1, 1

In [48]:
class Inference(object):
    def __init__(self, model, vectorizer):
        self.model = model
        self.model.to("cpu")
        self.vectorizer = vectorizer
  
    def predict_category(self, image):
        # Vectorize
        image_vector = self.vectorizer.vectorize(image)
        image_vector = torch.tensor(image_vector).unsqueeze(0)
        
        # Forward pass
        self.model.eval()
        y_pred = self.model(x=image_vector, apply_softmax=True)

        # Top category
        y_prob, indices = y_pred.max(dim=1)
        index = indices.item()

        # Predicted category
        category = vectorizer.category_vocab.lookup_index(index)
        probability = y_prob.item()
        return {'category': category, 'probability': probability}
    
    def predict_top_k(self, image, k):
        # Vectorize
        image_vector = self.vectorizer.vectorize(image)
        image_vector = torch.tensor(image_vector).unsqueeze(0)
        
        # Forward pass
        self.model.eval()
        y_pred = self.model(x=image_vector, apply_softmax=True)
        
        # Top k categories
        y_prob, indices = torch.topk(y_pred, k=k)
        probabilities = y_prob.detach().numpy()[0]
        indices = indices.detach().numpy()[0]

        # Results
        results = []
        for probability, index in zip(probabilities, indices):
            category = self.vectorizer.category_vocab.lookup_index(index)
            results.append({'category': category, 'probability': probability})

        return results

In [62]:
# Get a sample
sample = split_df[split_df.split=="test"].iloc[1000]

In [63]:
# Inference
inference = Inference(model=model, vectorizer=vectorizer)
prediction = inference.predict_category(sample.image)
print ("Actual:", sample.category)
test_image = np.array(sample.image, dtype='uint8')
plt.imshow(test_image)
plt.axis("off")
print("({} → p={:0.2f})".format(prediction['category'], 
                                prediction['probability']))

Actual: car
(car → p=1.00)


<Figure size 72x72 with 1 Axes>

In [64]:
# # Top-k inference
top_k = inference.predict_top_k(sample.image, k=len(vectorizer.category_vocab))
print ("Actual:", sample.category)
plt.imshow(test_image)
plt.axis("off")
for result in top_k:
    print ("{} → (p={:0.2f})".format(result['category'], 
                                     result['probability']))

Actual: car
car → (p=1.00)
truck → (p=0.00)
ship → (p=0.00)
plane → (p=0.00)
frog → (p=0.00)
dog → (p=0.00)
cat → (p=0.00)
bird → (p=0.00)
deer → (p=0.00)
horse → (p=0.00)


<Figure size 72x72 with 1 Axes>